In [1]:
from typing import List, Optional
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: List[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

In [4]:
# Problem Category: Easy
# Link: https://leetcode.com/problems/find-mode-in-binary-search-tree/submissions/1187947888
class Solution:
    def findMode(self, root: BinaryTreeNode) -> List[int]:
        def backtrack(curr: BinaryTreeNode):
            if curr == None:
                return
            else:
                hm[curr.val] = hm.get(curr.val, 0) + 1
                backtrack(curr.left)
                backtrack(curr.right)

        hm: dict[int, int] = dict()
        backtrack(root)
        max_ = max(hm.values())
        result: list[int] = []
        for k, v in hm.items():
            if v == max_:
                result.append(k)
        return result

# Testing the solution
assert Solution().findMode(BinaryTreeNode.from_array([1,None,2,2])) == [2]
assert Solution().findMode(BinaryTreeNode.from_array([0])) == [0]

In [5]:
# Problem Category: Medium
# Link: https://leetcode.com/problems/find-bottom-left-tree-value/submissions/1188894879/
class Solution:
    def findBottomLeftValue(self, root: BinaryTreeNode) -> int:
        next_: List[BinaryTreeNode] = [root]
        while True:
            next_next_: List[BinaryTreeNode] = []
            for node in next_:
                if node.left:
                    next_next_.append(node.left)
                if node.right:
                    next_next_.append(node.right)
            if next_next_:
                next_ = next_next_
            else:
                break
        return next_[0].val

# Testing the solution
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([2, 1, 3])) == 1
assert Solution().findBottomLeftValue(BinaryTreeNode.from_array([1,2,3,4,None,5,6,None,None,7])) == 7